# Чат-клієнт з використанням Tkinter та сокетів

## Мета
Створити настільний додаток на Python за допомогою Tkinter та сокетів для реалізації функціоналу чат-клієнта та сервера.

## Основні вимоги

### Інтерфейс користувача
- Створіть вікно додатку з використанням Tkinter для клієнта.
- Реалізуйте текстове поле для введення повідомлень і кнопку надсилання.
- Відображайте вхідні та вихідні повідомлення в області перегляду чату.
- Інтерфейс має бути адаптивним *(адекватно виглядати на різних розмірах вікна)*

### Мережева взаємодія
- Реалізуйте базовий сервер на сокетах, який може приймати повідомлення від клієнтів і пересилати їх усім підключеним клієнтам.
- Клієнт повинен підключатися до сервера і надсилати/отримувати повідомлення через сокети.

### Додаткове завдання
- Реалізуйте вибір імені користувача та відображайте його в чаті.
- Додайте логування усіх повідомлень у текстовий файл як на боці клієнта, так і сервера.
- Забезпечте можливість підключення декількох клієнтів до сервера одночасно.

## Критерії оцінки
- Стабільність з'єднання та коректність роботи мережевого взаємодії.
- Якість та зручність користувацького інтерфейсу.
- Реалізація багатокористувацького підключення та додаткових функцій.

## Приклад коду

### Приклад серверу:

```python
import socket
import threading

def client_thread(conn, addr):
    print(f"Connected by {addr}")
    while True:
        message = conn.recv(1024).decode()
        if not message:
            break
        print(f"Received from {addr}: {message}")
    conn.close()

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 12345))
    server.listen()

    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=client_thread, args=(conn, addr))
        thread.start()

if __name__ == "__main__":
    start_server()
```

### Приклад клієнта:

```python
import socket

def send_message():
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('localhost', 12345))

    while True:
        message = input("Your message: ")
        if message.lower() == 'quit':
            break
        client.send(message.encode())

    client.close()

if __name__ == "__main__":
    send_message()
```


In [1]:
import tkinter as tk
import socket
import threading

class ChatClient:
    def __init__(self, root):
        self.root = root
        self.nickname = None

        self.root.title("Chat Client")

        self.frame = tk.Frame(self.root)

        self.my_message = tk.StringVar()
        self.my_message.set("")

        self.scrollbar = tk.Scrollbar(self.frame)
        self.chat_box = tk.Text(self.frame, height=15, width=50, yscrollcommand=self.scrollbar.set)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.chat_box.pack(side=tk.LEFT, fill=tk.BOTH)
        self.chat_box.pack()
        self.frame.pack()

        self.entry_field = tk.Entry(self.root, textvariable=self.my_message)
        self.entry_field.bind("<Return>", self.send_message)
        self.entry_field.pack()

        self.send_button = tk.Button(self.root, text="Send", command=self.send_message)
        self.send_button.pack()

        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        self.receive_thread = threading.Thread(target=self.receive_messages)
        self.receive_thread.start()

    def on_closing(self):
        self.my_message.set("quit")
        self.send_message()
        self.root.destroy()

    def send_message(self, event=None):
        message = self.my_message.get()
        self.my_message.set("")

        if not self.nickname:
            self.nickname = message
            message = f"Your nickname is set to: {self.nickname}"
            self.chat_box.insert(tk.END, message + '\n')
        else:
            self.client_socket.send(bytes(message, 'utf-8'))

        if message.lower() == 'quit':
            self.client_socket.close()
            self.root.quit()

    def receive_messages(self):
        self.client_socket.connect(('localhost', 12345))
        while True:
            try:
                message = self.client_socket.recv(1024).decode('utf-8')
                if message:
                    self.chat_box.insert(tk.END, message + '\n')
            except OSError:
                break

    def start(self):
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.root.mainloop()


def start_server():
    clients = []
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 12345))
    server.listen()

    while True:
        conn, addr = server.accept()
        clients.append(conn)
        thread = threading.Thread(target=client_thread, args=(conn, addr, clients))
        thread.start()

def client_thread(conn, addr, clients):
    while True:
        try:
            message = conn.recv(1024).decode()
            if not message:
                break
            print(f"Received from {addr}: {message}")

            for client in clients:
                if client != conn:
                    client.send(message.encode())

        except ConnectionResetError:
            print(f"Connection with {addr} closed")
            break


if __name__ == "__main__":
    server_thread = threading.Thread(target=start_server)
    server_thread.start()

    root = tk.Tk()
    chat_client = ChatClient(root)
    chat_client.start()


Received from ('127.0.0.1', 51553): hello
Received from ('127.0.0.1', 51553): bebra
Received from ('127.0.0.1', 51553): s
Received from ('127.0.0.1', 51553): bye
Received from ('127.0.0.1', 51553): quit
Connection with ('127.0.0.1', 51553) closed
